In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [22]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share','PPO Share','HMO Share']], 
                right_index=True, left_on = 'Market_ID')

data['Nest Market Size'] = ( (data['Inside Good Share'] * data['PPO']) / data['PPO Share'] 
                            + (data['Inside Good Share'] * (1 - data['PPO'])) /data['HMO Share'])

#calculate ln(Inside Good Share)
data['ln(Inside Good Share)'] = np.log( data['Inside Good Share']/data['Nest Market Size'] ) 
data = data.fillna(0) #necessary for log(1), unsure why

print data.min()

Market_ID                1.000000
Plan_ID                  1.000000
PPO                      0.000000
Network Score            0.780000
Satisfaction Score       0.725000
Premium                  2.269728
Inside Good Share        0.037670
Outside Good Share       0.096070
PPO Share                0.000000
HMO Share                0.000000
Nest Market Size         0.000000
ln(Inside Good Share)   -2.428602
dtype: float64


In [23]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
#hausman_instr = pd.concat([mkt_dum,plan_dum],axis=1)
hausman_instr = plan_dum

In [24]:
#set up data for logit
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','Premium','PPO']]

# add ln(inside good share) as regressor like formula
x_nested = data[['Network Score','Satisfaction Score','Premium','PPO','ln(Inside Good Share)']]

In [25]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g 

    
#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model1 = logit(y , x, hausman_instr)

result1 = model1.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result1.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 292
         Function evaluations: 497
Optimization terminated successfully.
         Current function value: 0.005149
         Iterations: 139
         Function evaluations: 249
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.99
Model:                                                   logit   Prob (Hansen J):                 0.108
Method:                                                    GMM                                         
Date:                                         Fri, 05 Oct 2018                                         
Time:                                                 14:57:04                                         
No. Observations:                                         3300                                  

In [26]:
#set up and run model
beta_nested = np.full(len(x_nested.columns),1)
model2 = logit(y , x_nested, hausman_instr)
result2 = model2.fit(beta_nested, maxiter=2, optim_method='nm', wargs=dict(centered=False))

print(result2.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 524
         Function evaluations: 850
Optimization terminated successfully.
         Current function value: 0.004920
         Iterations: 153
         Function evaluations: 267
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.24
Model:                                                   logit   Prob (Hansen J):                0.0931
Method:                                                    GMM                                         
Date:                                         Fri, 05 Oct 2018                                         
Time:                                                 14:57:06                                         
No. Observations:                                         3300                                  

In [46]:
#compute exp(delta_j)
delta = data.copy()[['Market_ID','PPO','Premium']]

x = np.array(x)
beta1 = result1.params

delta['exp_delta'] = np.exp(np.matmul(x,beta1))

#compute 1 + sum_j exp(delta_j)
sum_delta = delta.groupby('Market_ID').sum()
sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 

delta = pd.merge(delta, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

#compute s_j
delta['fitted_share'] = delta['exp_delta']/delta['sum_exp_delta']

#compute cross price
shares = np.array([delta['fitted_share']]).transpose()
price = np.array([delta['Premium']])

elasticity = np.matmul(shares * beta1[3],price )

print shares.shape
print price.shape
print elasticity.shape

#correct diagonal for own price
for i in range(len(shares)):
    elasticity[i][i] = (1-shares[i][0])* beta1[3] * price[0][i]

(3300, 1)
(1, 3300)
(3300, 3300)
